## Yahoo Fundamental

In [ ]:
def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
    """
    Extrat data from trading economics callendar.

    Parameters:
        classe_tabela (str): Class CSS from table to scrap (optional).

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        if table_class:
            tabela = soup.find('table', {'class': table_class})
        else:
            tabela = soup.find('table')
        
        headers = [th.text.strip() for th in tabela.find_all('th')]
        print(headers)
        rows = []
        for row in tabela.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            if cols: 
                rows.append(cols)

        df = pd.DataFrame(rows, columns=headers if headers else None)
        

    except requests.exceptions.RequestException as e:
        print(f"Error accessing URL: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

    return df


In [ ]:
def get_whalewisdom(ticker: str) -> pd.DataFrame:
    """
    Extracts data from WhaleWisdom for a given stock ticker.

    Parameters:
        ticker (str): Stock ticker symbol.

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encontrar a div com a classe "v-data-table__wrapper"
        div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
        if not div_wrapper:
            raise ValueError("Div com a classe 'v-window__container' não encontrada.")

        # Buscar a primeira tabela dentro desta div
        tabela = div_wrapper.find('table')
        if not tabela:
            raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
        # Coletar os cabeçalhos da tabela ignorando ícones
        headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
        # print(headers)

        # Encontrando todas as linhas dentro da tabela (tbody > tr)
        rows = tabela.find("tbody").find_all("tr")
        print(rows)
        # Extraindo os dados de cada linha
        data = []
        for row in rows:
            columns = row.find_all("td")

            # Pegando os valores correspondentes
            institution_name = columns[0].text.strip()
            shares_held = columns[2].text.strip()
            portfolio_value = columns[3].text.strip()
            percentage_ownership = columns[4].text.strip()
            last_report_date = columns[-1].text.strip()

            data.append({
                "Instituição": institution_name,
                "Ações Detidas": shares_held,
                "Valor do Portfólio": portfolio_value,
                "Percentual de Participação": percentage_ownership,
                "Última Atualização": last_report_date
            })
        #print(data)
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
    except ValueError as e:
        print(f"Erro na extração de dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")
    
    return pd.DataFrame()

In [ ]:
df_whale_s = get_whalewisdom(ticker = "ctre")
df_whale_s

In [ ]:
base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json"
}

response = requests.get(base_url, headers=headers)
response.raise_for_status()

response.text

In [ ]:
import hashlib
import hmac
import base64
import json
import time
import urllib.parse
import requests

class WhaleWisdom:
    def __init__(self, secret_key, shared_key):
        self.secret_key = secret_key
        self.shared_key = shared_key
        self.timestamp = self.get_timestamp()

    def get_timestamp(self):
        """Gera o timestamp no formato UTC"""
        return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

    def signature(self, args):
        """Gera a assinatura HMAC-SHA1 em Base64"""
        message = f"{args}\n{self.timestamp}".encode('utf-8')
        hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
        return base64.b64encode(hmac_digest).decode('utf-8')

    def encode(self, string):
        """Codifica a string para URL"""
        return urllib.parse.quote(string, safe='')

    def endpoint(self, args):
        """Constrói a URL da API do WhaleWisdom"""
        encoded_args = self.encode(json.dumps(args))
        api_sig = self.signature(json.dumps(args))
        encoded_timestamp = self.encode(self.timestamp)

        return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

    def request(self, args):
        """Faz a requisição à API"""
        url = self.endpoint(args)
        response = requests.get(url)

        if response.status_code == 200:
            return response.text
        else:
            print(f"Erro {response.status_code}: {response.text}")
            return None


# **Exemplo de Uso**
WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# Obter as participações (holdings) de um gestor específico
response = ww.request({
    "command": "stock_lookup",
    "symbol": "ctre"
})

data = json.loads(response)

stock_id = data["stocks"][0]["id"]
name = data["stocks"][0]["name"]
status = data["stocks"][0]["status"]
link = data["stocks"][0]["link"]

stock_id

In [ ]:
response = ww.request(
{"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
response


In [ ]:
response

## STOCKS



In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import math
from datetime import datetime, timedelta
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler


# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários


In [2]:
def get_yoy_metric(series):
        if isinstance(series, pd.Series):
            series = series.replace(0, np.nan).dropna()

            if len(series) >= 2:
                yoy_growth = ((series - series.shift(-1)) / series.shift(-1)) * 100
                yoy_growth = yoy_growth.dropna()
            else:
                yoy_growth = pd.Series("N/A", index=series.index)

        return yoy_growth

def get_cagr_metric(series):
    if isinstance(series, pd.Series):
        series = pd.to_numeric(series, errors='coerce').dropna()

        if len(series) >= 2:
            start_value = series.iloc[-1]
            end_value = series.iloc[0]
            num_years = len(series) - 1

            if start_value != 0:
                cagr = ((end_value / start_value) ** (1 / num_years)) - 1
            else:
                cagr = "N/A"
        else:
            cagr = "N/A"
    else:
        cagr = "N/A"

    cagr_percent = cagr * 100 if cagr != "N/A" else "N/A"

    return cagr_percent


In [3]:
def get_sector_etf_info(sector : str, search_value : str = "info"):
    """
    Return information about symbol ETF sector.
    """
    sector_map = {
        "Technology": "XLK",
        "Financial Services": "XLF",
        "Consumer Cyclical": "XLY",
        "Healthcare": "XLV",
        "Communication Services": "XLC",
        "Industrials": "XLI",
        "Consumer Defensive": "XLP",
        "Energy": "XLE",
        "Real Estate": "XLRE",
        "Basic Materials": "XLB",
        "Utilities": "XLU",
    }

    etf_symbol = sector_map.get(sector)
    if not etf_symbol:
        return "N/A"

    ticker = yf.Ticker(etf_symbol)

    if search_value == "info":
        return ticker.info
    else:
        return ticker.info.get(search_value, "N/A")

In [4]:
def get_symbol_fundamental_info(symbol : str):
        '''
        Return detailed fundamental information about asset
        '''
        try:
            yahoo_symbol_info = yf.Ticker(symbol).info
        except:
            yahoo_symbol_info = {}
        try:
            yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
        except:
            yahoo_symbol_balancesheet = pd.DataFrame()
        try:
            yahoo_symbol_income = yf.Ticker(symbol).income_stmt
        except:
            yahoo_symbol_income = pd.DataFrame()
        try:
            yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
        except:
            yahoo_symbol_cashflow = pd.DataFrame()
        
        # Valuation
        sector = yahoo_symbol_info.get("sector")

        sector_pe = get_sector_etf_info(sector, "trailingPE")

        # Dividends & BuyBacks
        eps_ann = yahoo_symbol_info.get("epsCurrentYear")
        dividend_rate = yahoo_symbol_info.get("dividendRate")

        if (
            eps_ann is not None and
            dividend_rate is not None and
            dividend_rate != 0 and
            not math.isnan(eps_ann) and
            not math.isnan(dividend_rate)
        ):
            div_coverage_rate = eps_ann / dividend_rate
        else:
            div_coverage_rate = None

        # Profitability
        # net_income
        if 'Net Income' in yahoo_symbol_income.index:
            net_income = yahoo_symbol_income.loc['Net Income']
            if pd.isna(net_income).all():
                net_income = None

        # total_revenue
        if 'Total Revenue' in yahoo_symbol_income.index:
            total_revenue = yahoo_symbol_income.loc['Total Revenue']
            if pd.isna(total_revenue).all():
                total_revenue = None

        # cost_of_revenue
        if 'Cost Of Revenue' in yahoo_symbol_income.index:
            cost_of_revenue = yahoo_symbol_income.loc['Cost Of Revenue']
            if pd.isna(cost_of_revenue).all():
                cost_of_revenue = None

        # gross_profit
        if 'Gross Profit' in yahoo_symbol_income.index:
            gross_profit = yahoo_symbol_income.loc['Gross Profit']
            if pd.isna(gross_profit).all():
                gross_profit = None

        # operating_expenses
        if 'Operating Expense' in yahoo_symbol_income.index:
            operating_expenses = yahoo_symbol_income.loc['Operating Expense']
            if pd.isna(operating_expenses).all():
                operating_expenses = None
                    

        
        yoy_cost_of_revenue = get_yoy_metric(cost_of_revenue)
        yoy_total_revenue = get_yoy_metric(total_revenue)
        yoy_operating_expenses = get_yoy_metric(operating_expenses)

        cagr_cost_of_revenue = get_cagr_metric(cost_of_revenue)
        cagr_total_revenue = get_cagr_metric(total_revenue)
        cagr_operating_expenses = get_cagr_metric(operating_expenses)

        # Growth & NetWorth & Health
        # total_assets
        if 'Total Assets' in yahoo_symbol_balancesheet.index:
            total_assets = yahoo_symbol_balancesheet.loc['Total Assets']
            if pd.isna(total_assets).all():
                total_assets = None
        
        # current_liabilities
        if 'Current Liabilities' in yahoo_symbol_balancesheet.index:
            current_liabilities = yahoo_symbol_balancesheet.loc['Current Liabilities']
            if pd.isna(current_liabilities).all():
                current_liabilities = None

        # non_current_liabilities
        if 'Total Non Current Liabilities Net Minority Interest' in yahoo_symbol_balancesheet.index:
            non_current_liabilities = yahoo_symbol_balancesheet.loc['Total Non Current Liabilities Net Minority Interest']
            if pd.isna(non_current_liabilities).all():
                non_current_liabilities = None

        # total_liabilities
        if not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
            total_liabilities = (current_liabilities + non_current_liabilities)
        else:
            total_liabilities = None

        # net_worth
        if not total_assets.isna().all() and not current_liabilities.isna().all() and not non_current_liabilities.isna().all():
            total_assets = total_assets.replace(0, np.nan)
            net_worth = total_assets.iloc[0] - total_liabilities.iloc[0]
        else:
            net_worth = None

        # current_assets
        if 'Current Assets' in yahoo_symbol_balancesheet.index:
            current_assets = yahoo_symbol_balancesheet.loc['Current Assets']
            if pd.isna(current_assets).all():
                current_assets = None

        # non_current_assets
        if 'Total Non Current Assets' in yahoo_symbol_balancesheet.index:
            non_current_assets = yahoo_symbol_balancesheet.loc['Total Non Current Assets']
            if pd.isna(non_current_assets).all():
                non_current_assets = None
        
        # short_term_debt_coverage
        if not current_assets.isna().all() and not current_liabilities.isna().all():
            short_term_debt_coverage = current_assets - current_liabilities
        else:
            short_term_debt_coverage = None

        # long_term_debt_coverage
        if not non_current_assets.isna().all() and not non_current_liabilities.isna().all():
            long_term_debt_coverage = non_current_assets - non_current_liabilities
        else:
            long_term_debt_coverage = None

        cagr_total_liabilities = get_cagr_metric(total_liabilities)
        cagr_total_assets = get_cagr_metric(total_assets)

        # cash_cash_equivalents
        if 'Cash And Cash Equivalents' in yahoo_symbol_balancesheet.index:
            cash_cash_equivalents = yahoo_symbol_balancesheet.loc['Cash And Cash Equivalents']
            if pd.isna(cash_cash_equivalents).all():
                cash_cash_equivalents = None

        # stockholders_equity
        if 'Stockholders Equity' in yahoo_symbol_balancesheet.index:
            stockholders_equity = yahoo_symbol_balancesheet.loc['Stockholders Equity']
            if pd.isna(stockholders_equity).all():
                stockholders_equity = None
        
        cagr_stockholder_equity = get_cagr_metric(stockholders_equity)
        
        # Cashflow
        # free_cashflow
        if 'Free Cash Flow' in yahoo_symbol_cashflow.index:
            free_cashflow = yahoo_symbol_cashflow.loc['Free Cash Flow']
            if pd.isna(free_cashflow).all():
                free_cashflow = None
        
        # operating_cashflow
        if 'Operating Cash Flow' in yahoo_symbol_cashflow.index:
            operating_cashflow = yahoo_symbol_cashflow.loc['Operating Cash Flow']
            if pd.isna(operating_cashflow).all():
                operating_cashflow = None

        # capital_expenditure
        if 'Capital Expenditure' in yahoo_symbol_cashflow.index:
            capital_expenditure = yahoo_symbol_cashflow.loc['Capital Expenditure']
            if pd.isna(capital_expenditure).all():
                capital_expenditure = None

        # market_cap
        market_cap = yahoo_symbol_info.get('marketCap')
        if market_cap is None or (isinstance(market_cap, float) and math.isnan(market_cap)):
            market_cap = None

        # free_cashflow_yield
        if not pd.isna(market_cap) and market_cap != 0 and not free_cashflow.isna().all():
            free_cashflow_yield = ((free_cashflow.iloc[0] / market_cap) * 100)
        else:
            free_cashflow_yield = None

        # Ratios
        # current_ratio
        if not current_assets.isna().all() and not current_liabilities.isna().all():
            current_liabilities = current_liabilities.replace(0, np.nan)
            current_ratio_series = (current_assets / current_liabilities) * 100
            current_ratio_series = current_ratio_series.replace([np.inf, -np.inf], np.nan)

            if current_ratio_series.isna().all():
                current_ratio = None
            else:
                current_ratio = current_ratio_series
        else:
            current_ratio = None

        
        # cagr_current_ratio
        cagr_current_ratio = get_cagr_metric(current_ratio)


        # cash_ratio
        if not cash_cash_equivalents.isna().all() and not current_liabilities.isna().all():
            current_liabilities = current_liabilities.replace(0, np.nan)
            cash_ratio_series = (cash_cash_equivalents / current_liabilities) * 100
            cash_ratio_series = cash_ratio_series.replace([np.inf, -np.inf], np.nan)

            if cash_ratio_series.isna().all():
                cash_ratio = None
            else:
                cash_ratio = cash_ratio_series
        else:
            cash_ratio = None

        
        # cagr_cash_ratio
        cagr_cash_ratio = get_cagr_metric(cash_ratio)

        # gross_margin
        if not gross_profit.isna().all() and not total_revenue.isna().all():
            total_revenue = total_revenue.replace(0, np.nan)
            gross_margin_series = (gross_profit / total_revenue) * 100
            gross_margin_series = gross_margin_series.replace([np.inf, -np.inf], np.nan)

            if gross_margin_series.isna().all():
                gross_margin = None
            else:
                gross_margin = gross_margin_series
        else:
            gross_margin = None


        # cagr_gross_margin
        # cagr_gross_margin = get_cagr_metric(gross_margin)

        # operating_income
        if 'Operating Income' in yahoo_symbol_income.index:
            operating_income = yahoo_symbol_income.loc['Operating Income']
            if pd.isna(operating_income).all():
                operating_income = None

        # operation_margin
        if not operating_income.isna().all() and not total_revenue.isna().all():
            total_revenue = total_revenue.replace(0, np.nan)
            operating_margin_series = (operating_income / total_revenue) * 100
            operating_margin_series = operating_margin_series.replace([np.inf, -np.inf], np.nan)

            if operating_margin_series.isna().all():
                operating_margin = None
            else:
                operating_margin = operating_margin_series
        else:
            operating_margin = None


        # cagr_operating_margin
        cagr_operating_margin = get_cagr_metric(operating_margin)

        # profit_margin
        if not net_income.isna().all() and not total_revenue.isna().all():
            total_revenue = total_revenue.replace(0, np.nan)
            profit_margin_series = (net_income / total_revenue) * 100
            profit_margin_series = profit_margin_series.replace([np.inf, -np.inf], np.nan)

            if profit_margin_series.isna().all():
                profit_margin = None
            else:
                profit_margin = profit_margin_series
        else:
            profit_margin = None

        # cagr_profit_margin
        cagr_profit_margin = get_cagr_metric(profit_margin)
        
        # return_on_equity
        if not net_income.isna().all() and not stockholders_equity.isna().all():
            stockholders_equity = stockholders_equity.replace(0, np.nan)
            return_on_equity_series = (net_income / stockholders_equity) * 100
            return_on_equity_series = return_on_equity_series.replace([np.inf, -np.inf], np.nan)

            if return_on_equity_series.isna().all():
                return_on_equity = None
            else:
                return_on_equity = return_on_equity_series
        else:
            return_on_equity = None

        # cagr_return_on_equity
        cagr_return_on_equity = get_cagr_metric(return_on_equity)
        
        yahoo_symbol_fundamental_info = {
            "valuation": {
                "trailingPE": yahoo_symbol_info.get("trailingPE", "N/A"),
                "sectorTrailingPE": sector_pe,
                "forwardPE": yahoo_symbol_info.get("forwardPE", "N/A"),
                "PEGRatio": yahoo_symbol_info.get("trailingPegRatio", "N/A"),
            },
            "dividends": {
                "divCoverageRate": div_coverage_rate,
                "dividendYield": yahoo_symbol_info.get("dividendYield", "N/A"),
                "fiveYearAvgDividendYield": yahoo_symbol_info.get("fiveYearAvgDividendYield", "N/A"),
            },
            "profitability": {
                "NetIncome": net_income.iloc[0],
                "TotalRevenue": total_revenue.iloc[0],
                "CostOfRevenue": cost_of_revenue.iloc[0],
                "GrossProfit": gross_profit.iloc[0],
                "OperatingExpenses": operating_expenses.iloc[0],
                "CostOfRevenueCAGR": cagr_cost_of_revenue,
                "TotalRevenueCAGR": cagr_total_revenue,
                "OperatingExpensesCAGR": cagr_operating_expenses,
                # "CostOfRevenueYOY": yoy_cost_of_revenue,
                # "TotalRevenueYOY": yoy_total_revenue,
            },
            "liquidity": {
                # Actual
                "TotalAssets": total_assets.iloc[0],
                "TotalLiabilities": total_liabilities.iloc[0],
                "NetWorth": net_worth,
                "CashCashEquivalents": cash_cash_equivalents.iloc[0],
                # Short Term 1y
                "ShortTermDebtCoverage" : short_term_debt_coverage.iloc[0],
                "CurrentAssets": current_assets.iloc[0],
                "CurrentLiabilities": current_liabilities.iloc[0],
                # Long Term
                "LongTermDebtCoverage" : long_term_debt_coverage.iloc[0],
                "NonCurrentAssets": non_current_assets.iloc[0],
                "NonCurrentLiabilities": non_current_liabilities.iloc[0],
                # Growth
                "TotalAssetsCAGR": cagr_total_assets,
                "TotalLiabilitiesCAGR": cagr_total_liabilities,
                # Stockholders Equity
                "StockholdersEquityCAGR": cagr_stockholder_equity,
                "StockholdersEquity": stockholders_equity.iloc[0],
            },
            "cashflow": {
                "FreeCashflow": free_cashflow.iloc[0],
                "OperatingCashflow": operating_cashflow.iloc[0],
                "CapitalExpenditure": capital_expenditure.iloc[0],
                "MarketCap": market_cap,
                "FreeCashflowYield": free_cashflow_yield,
            },
            "ratios": {
                # Health & Debt
                "CurrentRatio": current_ratio.iloc[0],
                "CurrentRatioCAGR": cagr_current_ratio,
                "CashRatio": cash_ratio.iloc[0],
                "CashRatioCAGR": cagr_cash_ratio,
                # Margins
                "GrossMargin": gross_margin.iloc[0],
                # "GrossMarginCAGR": cagr_gross_margin,
                "OperatingMargin": operating_margin.iloc[0],
                "OperatingMarginCAGR": cagr_operating_margin,
                "ProfitMargin": profit_margin.iloc[0],
                "ProfitMarginCAGR": cagr_profit_margin,
                "ReturnOnEquity": return_on_equity.iloc[0],
                "ReturnOnEquityCAGR": cagr_return_on_equity,
            },
            "market_risk_and_sentiment": {
                "beta": yahoo_symbol_info.get("beta", "N/A"),
                "auditRisk": yahoo_symbol_info.get("auditRisk", "N/A"),
                "boardRisk": yahoo_symbol_info.get("boardRisk", "N/A"),
                "sharesPercentSharesOut": yahoo_symbol_info.get("sharesPercentSharesOut", "N/A"),
                "recommendationMean": yahoo_symbol_info.get("recommendationMean", "N/A"),
                "targetMeanPrice": yahoo_symbol_info.get("targetMeanPrice", "N/A")
            }
        }
        return yahoo_symbol_fundamental_info

In [5]:
metrics = get_symbol_fundamental_info(symbol)

In [6]:
metrics

{'valuation': {'trailingPE': 28.47887,
  'sectorTrailingPE': 28.72205,
  'forwardPE': 21.556208,
  'PEGRatio': 1.7483},
 'dividends': {'divCoverageRate': 7.28687,
  'dividendYield': 0.53,
  'fiveYearAvgDividendYield': 0.57},
 'profitability': {'NetIncome': 93736000000.0,
  'TotalRevenue': 391035000000.0,
  'CostOfRevenue': 210352000000.0,
  'GrossProfit': 180683000000.0,
  'OperatingExpenses': 57467000000.0,
  'CostOfRevenueCAGR': -0.4131655096255016,
  'TotalRevenueCAGR': 2.247001899501888,
  'OperatingExpensesCAGR': 9.40257491775962},
 'liquidity': {'TotalAssets': 364980000000.0,
  'TotalLiabilities': 308030000000.0,
  'NetWorth': 56950000000.0,
  'CashCashEquivalents': 29943000000.0,
  'ShortTermDebtCoverage': -23405000000.0,
  'CurrentAssets': 152987000000.0,
  'CurrentLiabilities': 176392000000.0,
  'LongTermDebtCoverage': 80355000000.0,
  'NonCurrentAssets': 211993000000.0,
  'NonCurrentLiabilities': 131638000000.0,
  'TotalAssetsCAGR': 1.3101967061472708,
  'TotalLiabilitiesCAGR

In [7]:
def safe_round(value):
    try:
        return round(float(value), 2)
    except (ValueError, TypeError):
        return None

In [8]:
def evaluate_metrics(metrics):
        """
        Evaluate fundamental metrics.
        """

        evaluated_metrics = {}

        # Valuation
        valuation = metrics.get('valuation', {})
        trailing_pe = safe_round(valuation.get("trailingPE"))
        sector_pe   = safe_round(valuation.get("sectorTrailingPE"))
        forward_pe  = safe_round(valuation.get("forwardPE"))

        # Guardar os valores convertidos
        evaluated_metrics["trailingPE"] = trailing_pe if trailing_pe is not None else None
        evaluated_metrics["sectorTrailingPE"] = sector_pe if sector_pe is not None else None
        evaluated_metrics["forwardPE"] = forward_pe if forward_pe is not None else None

        # Só avalia o score se TODOS estiverem presentes
        if None not in (trailing_pe, sector_pe, forward_pe):
            score_pe = 0
            score_pe += 1 if trailing_pe < sector_pe else -1 if trailing_pe > sector_pe else 0
            score_pe += 1 if forward_pe < sector_pe else -1 if forward_pe > sector_pe else 0
            score_pe += 1 if trailing_pe > forward_pe else -1 if trailing_pe < forward_pe else 0

            evaluated_metrics["trailingPE_evaluation"] = {
                3:  "Very Low Undervalued",
                2:  "Low Undervalued",
                1:  "Undervalued",
                0:  "Neutral Valued",
                -1: "Overvalued",
                -2: "High Overvalued",
            }.get(score_pe, "Very High Overvalued")
        else:
            evaluated_metrics["trailingPE_evaluation"] = "No Data"

        # Valuation - PEG Ratio
        peg_raw = safe_round(metrics.get('valuation', {}).get("PEGRatio"))

        evaluated_metrics["PEGRatio"] = peg_raw if peg_raw is not None else None

        if peg_raw is None:
            evaluated_metrics["PEGRatio_evaluation"] = "No Data"
        else:
            if peg_raw < 1:
                evaluated_metrics["PEGRatio_evaluation"] = "Undervalued"
            elif peg_raw == 1:
                evaluated_metrics["PEGRatio_evaluation"] = "Neutral Valued"
            else:
                evaluated_metrics["PEGRatio_evaluation"] = "Overvalued"

        
        # Dividends - Dividend Coverage Ratio
        div_coverage_raw = safe_round(metrics.get('dividends', {}).get("divCoverageRate"))

        # Armazenar o valor bruto
        evaluated_metrics["divCoverageRate"] = div_coverage_raw if div_coverage_raw is not None else None

        # Avaliação
        if div_coverage_raw is None:
            evaluated_metrics["divCoverageRate_evaluation"] = "No Data"
        else:
            if div_coverage_raw <= 1:
                evaluated_metrics["divCoverageRate_evaluation"] = "No Coverage"
            elif div_coverage_raw <= 1.5:
                evaluated_metrics["divCoverageRate_evaluation"] = "Bad Coverage (Cut)"
            elif div_coverage_raw <= 3:
                evaluated_metrics["divCoverageRate_evaluation"] = "Good Coverage"
            else:
                evaluated_metrics["divCoverageRate_evaluation"] = "Very Good Coverage (Greedy)"

        
        # Profitability - Cost of Revenue CAGR
        cost_revenue_cagr = safe_round(metrics.get('profitability', {}).get("CostOfRevenueCAGR"))

        # Armazenar o valor bruto
        evaluated_metrics["CostOfRevenueCAGR"] = cost_revenue_cagr if cost_revenue_cagr is not None else None

        # Avaliação
        if cost_revenue_cagr is None or math.isnan(cost_revenue_cagr):
            evaluated_metrics["CostOfRevenueCAGR_evaluation"] = "No Data"
        else:
            if cost_revenue_cagr <= 0:
                evaluated_metrics["CostOfRevenueCAGR_evaluation"] = "Good"
            else:
                evaluated_metrics["CostOfRevenueCAGR_evaluation"] = "Not Good"


        total_revenue_cagr = safe_round(metrics.get('profitability', {}).get("TotalRevenueCAGR"))
        evaluated_metrics["TotalRevenueCAGR"] = total_revenue_cagr if total_revenue_cagr is not None else None

        if total_revenue_cagr is None or math.isnan(total_revenue_cagr):
            evaluated_metrics["TotalRevenueCAGR_evaluation"] = "No Data"
        else:
            evaluated_metrics["TotalRevenueCAGR_evaluation"] = "Good" if total_revenue_cagr > 0 else "Not Good"

        # Debt
        net_worth = safe_round(metrics.get('liquidity', {}).get("NetWorth"))
        evaluated_metrics["NetWorth"] = net_worth if net_worth is not None else None

        if net_worth is None:
            evaluated_metrics["NetWorth_evaluation"] = "No Data"
        else:
            evaluated_metrics["NetWorth_evaluation"] = "Good" if net_worth > 0 else "Not Good (In Debt)"


            # Short Term Debt Coverage 
        short_debt_cov = safe_round(metrics.get('liquidity', {}).get("ShortTermDebtCoverage"))
        evaluated_metrics["ShortTermDebtCoverage"] = short_debt_cov if short_debt_cov is not None else None

        if short_debt_cov is None:
            evaluated_metrics["ShortTermDebtCoverage_evaluation"] = "No Data"
        else:
            evaluated_metrics["ShortTermDebtCoverage_evaluation"] = "Good" if short_debt_cov > 0 else "Not Good (In Debt)"

    
            # Long Term Debt Coverage 
        long_debt_cov = safe_round(metrics.get('liquidity', {}).get("LongTermDebtCoverage"))
        evaluated_metrics["LongTermDebtCoverage"] = long_debt_cov if long_debt_cov is not None else None

        if long_debt_cov is None:
            evaluated_metrics["LongTermDebtCoverage_evaluation"] = "No Data"
        else:
            evaluated_metrics["LongTermDebtCoverage_evaluation"] = "Good" if long_debt_cov > 0 else "Not Good (In Debt)"


            # Assets Growth
        total_assets_cagr = safe_round(metrics.get('liquidity', {}).get("TotalAssetsCAGR"))
        evaluated_metrics["TotalAssetsCAGR"] = total_assets_cagr if total_assets_cagr is not None else None

        if total_assets_cagr is None or math.isnan(total_assets_cagr):
            evaluated_metrics["TotalAssetsCAGR_evaluation"] = "No Data"
        else:
            evaluated_metrics["TotalAssetsCAGR_evaluation"] = "Good" if total_assets_cagr > 0 else "Not Good"


            # Liabilities Growth
        total_liabilities_cagr = safe_round(metrics.get('liquidity', {}).get("TotalLiabilitiesCAGR"))
        evaluated_metrics["TotalLiabilitiesCAGR"] = total_liabilities_cagr if total_liabilities_cagr is not None else None

        if total_liabilities_cagr is None or math.isnan(total_liabilities_cagr):
            evaluated_metrics["TotalLiabilitiesCAGR_evaluation"] = "No Data"
        else:
            evaluated_metrics["TotalLiabilitiesCAGR_evaluation"] = "Good" if total_liabilities_cagr <= 0 else "Not Good"


            # Stockholders Equity
        stockholders_equity_cagr = safe_round(metrics.get('liquidity', {}).get("StockholdersEquityCAGR"))
        evaluated_metrics["StockholdersEquityCAGR"] = stockholders_equity_cagr if stockholders_equity_cagr is not None else None

        if stockholders_equity_cagr is None or math.isnan(stockholders_equity_cagr):
            evaluated_metrics["StockholdersEquityCAGR_evaluation"] = "No Data"
        else:
            evaluated_metrics["StockholdersEquityCAGR_evaluation"] = "Good" if stockholders_equity_cagr > 0 else "Not Good"


        # Cash
            # Free Cashflow Yield
        fcf_yield = safe_round(metrics.get('cashflow', {}).get("FreeCashflowYield"))
        evaluated_metrics["FreeCashflowYield"] = fcf_yield if fcf_yield is not None else None

        if fcf_yield is None:
            evaluated_metrics["FreeCashflowYield_evaluation"] = "No Data"
        elif fcf_yield <= 2:
            evaluated_metrics["FreeCashflowYield_evaluation"] = "Overvalued / Bad to Generate Cash"
        elif fcf_yield <= 5:
            evaluated_metrics["FreeCashflowYield_evaluation"] = "Healthy / Consistent to Generates Cash"
        else:
            evaluated_metrics["FreeCashflowYield_evaluation"] = "Undervalued / Highly Profitable"


        # Ratios
            # Current Ratio
        current_ratio = safe_round(metrics.get('ratios', {}).get("CurrentRatio"))
        evaluated_metrics["CurrentRatio"] = current_ratio if current_ratio is not None else None

        if current_ratio is None:
            evaluated_metrics["CurrentRatio_evaluation"] = "No Data"
        else:
            if current_ratio <= 100:
                evaluated_metrics["CurrentRatio_evaluation"] = "Not Good (In Debt)"
            elif current_ratio <= 120:
                evaluated_metrics["CurrentRatio_evaluation"] = "Tight Margin to Debt"
            elif current_ratio <= 200:
                evaluated_metrics["CurrentRatio_evaluation"] = "Good Debt Coverage"
            else:
                evaluated_metrics["CurrentRatio_evaluation"] = "Perfect Coverage (Double +)"


            # Current Ratio Growth
        current_ratio_cagr = safe_round(metrics.get('ratios', {}).get("CurrentRatioCAGR"))
        evaluated_metrics["CurrentRatioCAGR"] = current_ratio_cagr if current_ratio_cagr is not None else None

        if current_ratio_cagr is None or math.isnan(current_ratio_cagr):
            evaluated_metrics["CurrentRatioCAGR_evaluation"] = "No Data"
        else:
            evaluated_metrics["CurrentRatioCAGR_evaluation"] = "Good" if current_ratio_cagr > 0 else "Not Good"


            # Cash Ratio
        cash_ratio = safe_round(metrics.get('ratios', {}).get("CashRatio"))
        evaluated_metrics["CashRatio"] = cash_ratio if cash_ratio is not None else None

        if cash_ratio is None:
            evaluated_metrics["CashRatio_evaluation"] = "No Data"
        else:
            if cash_ratio <= 50:
                evaluated_metrics["CashRatio_evaluation"] = "Not Good (In Debt)"
            elif cash_ratio <= 100:
                evaluated_metrics["CashRatio_evaluation"] = "Good Debt Coverage"
            else:
                evaluated_metrics["CashRatio_evaluation"] = "Good Debt Coverage (Too Conservative)"


            # Current Ratio Growth
        cash_ratio_cagr = safe_round(metrics.get('ratios', {}).get("CashRatioCAGR"))
        evaluated_metrics["CashRatioCAGR"] = cash_ratio_cagr if cash_ratio_cagr is not None else None

        if cash_ratio_cagr is None or math.isnan(cash_ratio_cagr):
            evaluated_metrics["CashRatioCAGR_evaluation"] = "No Data"
        else:
            if cash_ratio_cagr <= 0:
                evaluated_metrics["CashRatioCAGR_evaluation"] = "Not Good"
            else:
                evaluated_metrics["CashRatioCAGR_evaluation"] = "Good"


            # Gross Margin
        gross_margin = safe_round(metrics.get('ratios', {}).get("GrossMargin"))
        evaluated_metrics["GrossMargin"] = gross_margin if gross_margin is not None else None

        if gross_margin is None:
            evaluated_metrics["GrossMargin_evaluation"] = "No Data"
        else:
            if gross_margin <= 25:
                evaluated_metrics["GrossMargin_evaluation"] = "Not Good - Short Margins or High Costs"
            elif gross_margin <= 40:
                evaluated_metrics["GrossMargin_evaluation"] = "Healthy - Healthy Margins Good Management"
            else:
                evaluated_metrics["GrossMargin_evaluation"] = "Good - Efficient Costs Management"


            # Gross Margin Growth
        gross_margin_cagr = safe_round(metrics.get('ratios', {}).get("GrossMarginCAGR"))
        evaluated_metrics["GrossMarginCAGR"] = gross_margin_cagr if gross_margin_cagr is not None else None

        if gross_margin_cagr is None or math.isnan(gross_margin_cagr):
            evaluated_metrics["GrossMarginCAGR_evaluation"] = "No Data"
        else:
            evaluated_metrics["GrossMarginCAGR_evaluation"] = (
                "Good" if gross_margin_cagr > 0 else "Not Good"
            )


            # Operating Margin
        operating_margin = safe_round(metrics.get('ratios', {}).get("OperatingMargin"))
        evaluated_metrics["OperatingMargin"] = operating_margin if operating_margin is not None else None

        if operating_margin is None:
            evaluated_metrics["OperatingMargin_evaluation"] = "No Data"
        else:
            if operating_margin <= 10:
                evaluated_metrics["OperatingMargin_evaluation"] = "Not Good - High Operational Costs or Difficulties to Get Revenue"
            elif operating_margin <= 20:
                evaluated_metrics["OperatingMargin_evaluation"] = "Healthy - Healthy Operational Management"
            else:
                evaluated_metrics["OperatingMargin_evaluation"] = "Good - Efficient Cost Management"


            # Operational Margin Growth
        operating_margin_cagr = safe_round(metrics.get('ratios', {}).get("OperatingMarginCAGR"))
        evaluated_metrics["OperatingMarginCAGR"] = operating_margin_cagr if operating_margin_cagr is not None else None

        if operating_margin_cagr is None or math.isnan(operating_margin_cagr):
            evaluated_metrics["OperatingMarginCAGR_evaluation"] = "No Data"
        else:
            if operating_margin_cagr <= 0:
                evaluated_metrics["OperatingMarginCAGR_evaluation"] = "Not Good"
            else:
                evaluated_metrics["OperatingMarginCAGR_evaluation"] = "Good"

            
            # Profit Margin
        profit_margin = safe_round(metrics.get('ratios', {}).get("ProfitMargin"))
        evaluated_metrics["ProfitMargin"] = profit_margin if profit_margin is not None else None

        if profit_margin is None:
            evaluated_metrics["ProfitMargin_evaluation"] = "No Data"
        else:
            if profit_margin <= 5:
                evaluated_metrics["ProfitMargin_evaluation"] = "Not Good - High Operational Costs or Operational Problems"
            elif profit_margin <= 10:
                evaluated_metrics["ProfitMargin_evaluation"] = "Moderated - Potential but Need Improvements"
            elif profit_margin <= 20:
                evaluated_metrics["ProfitMargin_evaluation"] = "Healthy - Healthy and Solid Management"
            else:
                evaluated_metrics["ProfitMargin_evaluation"] = "Good - Highly Profitable and Eficient Profit Generate"


            # Profit Margin Growth
        profit_margin_cagr = safe_round(metrics.get('ratios', {}).get("ProfitMarginCAGR"))
        evaluated_metrics["ProfitMarginCAGR"] = profit_margin_cagr if profit_margin_cagr is not None else None

        if profit_margin_cagr is None or math.isnan(profit_margin_cagr):
            evaluated_metrics["ProfitMarginCAGR_evaluation"] = "No Data"
        else:
            if profit_margin_cagr <= 0:
                evaluated_metrics["ProfitMarginCAGR_evaluation"] = "Not Good - Declining or No Growth"
            else:
                evaluated_metrics["ProfitMarginCAGR_evaluation"] = "Good - Growing Profitability Over Time"

            
            # Return On Equity
        return_on_equity = safe_round(metrics.get('ratios', {}).get("ReturnOnEquity"))
        evaluated_metrics["ReturnOnEquity"] = return_on_equity if return_on_equity is not None else None

        if return_on_equity is None:
            evaluated_metrics["ReturnOnEquity_evaluation"] = "No Data"
        else:
            if return_on_equity <= 10:
                evaluated_metrics["ReturnOnEquity_evaluation"] = "Not Good - Low Efficiency on Equity Use"
            elif return_on_equity <= 15:
                evaluated_metrics["ReturnOnEquity_evaluation"] = "Moderated - Potential but Needs Improvement"
            elif return_on_equity <= 20:
                evaluated_metrics["ReturnOnEquity_evaluation"] = "Healthy - Efficient and Solid Management"
            else:
                evaluated_metrics["ReturnOnEquity_evaluation"] = "Good - Highly Efficient in Generating Profits"

            # Return On Equity Growth
        return_on_equity_cagr = safe_round(metrics.get('ratios', {}).get("ReturnOnEquityCAGR"))
        evaluated_metrics["ReturnOnEquityCAGR"] = return_on_equity_cagr if return_on_equity_cagr is not None else None

        if return_on_equity_cagr is None or math.isnan(return_on_equity_cagr):
            evaluated_metrics["ReturnOnEquityCAGR_evaluation"] = "No Data"
        else:
            if return_on_equity_cagr <= 0:
                evaluated_metrics["ReturnOnEquityCAGR_evaluation"] = "Not Good - No Growth or Negative Trend"
            else:
                evaluated_metrics["ReturnOnEquityCAGR_evaluation"] = "Good - Consistent Growth in Equity Returns"
        
        return evaluated_metrics if evaluated_metrics else "Indefinido"



In [9]:
evaluate_metrics(metrics)

{'trailingPE': 28.48,
 'sectorTrailingPE': 28.72,
 'forwardPE': 21.56,
 'trailingPE_evaluation': 'Very Low Undervalued',
 'PEGRatio': 1.75,
 'PEGRatio_evaluation': 'Overvalued',
 'divCoverageRate': 7.29,
 'divCoverageRate_evaluation': 'Very Good Coverage (Greedy)',
 'CostOfRevenueCAGR': -0.41,
 'CostOfRevenueCAGR_evaluation': 'Good',
 'TotalRevenueCAGR': 2.25,
 'TotalRevenueCAGR_evaluation': 'Good',
 'NetWorth': 56950000000.0,
 'NetWorth_evaluation': 'Good',
 'ShortTermDebtCoverage': -23405000000.0,
 'ShortTermDebtCoverage_evaluation': 'Not Good (In Debt)',
 'LongTermDebtCoverage': 80355000000.0,
 'LongTermDebtCoverage_evaluation': 'Good',
 'TotalAssetsCAGR': 1.31,
 'TotalAssetsCAGR_evaluation': 'Good',
 'TotalLiabilitiesCAGR': 2.28,
 'TotalLiabilitiesCAGR_evaluation': 'Not Good',
 'StockholdersEquityCAGR': -3.36,
 'StockholdersEquityCAGR_evaluation': 'Not Good',
 'FreeCashflowYield': 4.04,
 'FreeCashflowYield_evaluation': 'Healthy / Consistent to Generates Cash',
 'CurrentRatio': 86.7

news - news
sec_filings - Periodical reposts 

In [43]:
## INSIDER TRANSACTIONS ##

#  P - Purchase
#  S - Sale
#  A - Grant
#  D - Sale to Iss
#  G - Gift
#  F - Tax
#  M - Option Ex
#  X - Option Ex
#  C - Cnv Deriv
#  W - Inherited
#  No deriv	 Multiple Days

yahoo_symbol_insider_transactions = yf.Ticker(symbol).insider_transactions

yahoo_symbol_insider_transactions.drop(columns=['URL','Transaction'], inplace=True)
yahoo_symbol_insider_transactions[yahoo_symbol_insider_transactions['Start Date']>'2024-12-15']

,Shares,Value,Text,Insider,Position,Start Date,Ownership
0,108136,24184658.0,Sale at price 221.77 - 224.76 per share.,COOK TIMOTHY D,Chief Executive Officer,2025-04-02,D
1,35493,7950691.0,Sale at price 223.48 - 225.03 per share.,WILLIAMS JEFFREY E,Chief Operating Officer,2025-04-02,D
2,38822,8683252.0,Sale at price 221.68 - 224.62 per share.,ADAMS KATHERINE L,General Counsel,2025-04-02,D
3,218568,NaN,,COOK TIMOTHY D,Chief Executive Officer,2025-04-01,D
4,74535,NaN,,O'BRIEN DEIRDRE,Officer,2025-04-01,D
5,74535,NaN,,ADAMS KATHERINE L,General Counsel,2025-04-01,D
6,74535,NaN,,WILLIAMS JEFFREY E,Chief Operating Officer,2025-04-01,D
7,1516,343147.0,Sale at price 226.35 per share.,LEVINSON ARTHUR D,Director,2025-02-03,D
8,1516,NaN,,WAGNER SUSAN L,Director,2025-01-31,D
9,1516,NaN,,LEVINSON ARTHUR D,Director,2025-01-31,D


In [42]:
yahoo_symbol_insider_transactions['Ownership'].unique()

array(['D', 'I'], dtype=object)

In [ ]:
# def get_data_history(symbol : str, period : str, interval : str, start = '1900-01-01', end = datetime.now(), prepost : bool = True):
#         '''
#         Data collection from yahoo

#         Parameters:
#         period : str
#             Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
#         interval : str
#             Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
#         start: str
#             Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
#         end: str
#             Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
#         prepost : bool
#             Include Pre and Post market data in results? Default is False
#         '''
#         yahoo_data_history = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost)
#         yahoo_data_history.reset_index(inplace=True)

#         yahoo_data_history["Date"] = yahoo_data_history["Date"].dt.strftime("%Y-%m-%d %H:%M")

#         return yahoo_data_history

# data_history = get_data_history("", period, interval)

In [ ]:
# import plotly.graph_objects as go

# # Criar gráfico com Plotly
# fig = go.Figure()

# # Linha principal (Série Temporal)
# fig.add_trace(go.Scatter(
#     x=data_history['Date'], 
#     y=data_history['Close'], 
#     mode='lines', 
#     name='Preço de Fechamento'
# ))

# # Destacar Picos e Vales
# fig.add_trace(go.Scatter(
#     x=data_history['Date'].iloc[current_idx], 
#     y=current_pat, 
#     mode='markers', 
#     marker=dict(size=10, color='red'), 
#     name='Picos/Vales'
# ))

# # Configuração do layout
# fig.update_layout(
#     title='Detecção de Picos e Vales - Série Temporal',
#     xaxis_title='Data',
#     yaxis_title='Preço de Fechamento',
#     template='plotly_dark',  # Escolha visual moderna e clara
#     height=600,
#     width=1000
# )

# # Exibir gráfico
# fig.show()

In [ ]:
# def get_gartley_hp(moves, err_allowed, stop_factor, price_X):
#     XA, AB, BC, CD = moves

#     AB_range = np.array([.618 - err_allowed, .618 + err_allowed]) * abs(XA)
#     BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
#     CD_range = np.array([1.27 - err_allowed, 1.618 + err_allowed]) * abs(BC)

#     targets = {
#         "XA": None,
#         "AB": None,
#         "BC": None,
#         "CD": None,
#         "TP1": None,
#         "TP2": None,
#         "TP3": None,
#         "STOP": None
#     }

#     ## === Bullish Gartley === ##
#     if XA > 0 and AB < 0 and BC > 0 and CD < 0:
#         if AB_range[0] < abs(AB) < AB_range[1] and \
#            BC_range[0] < abs(BC) < BC_range[1] and \
#            CD_range[0] < abs(CD) < CD_range[1]:

#             D = XA + AB + BC + CD
#             C = XA + AB + BC
#             B = XA + AB

#             price_D = price_X + D
#             price_C = price_X + C
#             price_B = price_X + B

#             targets.update({
#                 "XA": XA,
#                 "AB": AB,
#                 "BC": BC,
#                 "CD": CD,
#                 "TP1": price_B,
#                 "TP2": price_C,
#                 "TP3": price_D + (price_C - price_D) * 1.618,
#                 "STOP": price_D - abs(CD) * stop_factor
#             })

#             return 1, targets  # Bullish

#     ## === Bearish Gartley === ##
#     elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
#         if AB_range[0] < abs(AB) < AB_range[1] and \
#            BC_range[0] < abs(BC) < BC_range[1] and \
#            CD_range[0] < abs(CD) < CD_range[1]:

#             D = XA + AB + BC + CD
#             C = XA + AB + BC
#             B = XA + AB

#             price_D = price_X + D
#             price_C = price_X + C
#             price_B = price_X + B

#             targets.update({
#                 "XA": XA,
#                 "AB": AB,
#                 "BC": BC,
#                 "CD": CD,
#                 "TP1": price_B,
#                 "TP2": price_C,
#                 "TP3": price_D - (price_D - price_C) * 1.618,
#                 "STOP": price_D + abs(CD) * stop_factor
#             })

#             return -1, targets  # Bearish

#     return np.nan, targets


In [ ]:
def get_bat_hp(moves, err_allowed, stop_factor, price_X):
    XA, AB, BC, CD = moves

    AB_range = np.array([.382 - err_allowed, .5 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.618 - err_allowed, 2.618 + err_allowed]) * abs(BC)

    targets = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None,
        "TP1": None,
        "TP2": None,
        "TP3": None,
        "STOP": None
    }

    ## === Bullish Bat === ##
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D + (price_C - price_D) * 1.618,
                "STOP": price_D - abs(CD) * stop_factor
            })

            return 1, targets  # Bullish

    ## === Bearish Bat === ##
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D - (price_D - price_C) * 1.618,
                "STOP": price_D + abs(CD) * stop_factor
            })

            return -1, targets  # Bearish

    return np.nan, targets


In [ ]:
def get_butterfly_hp(moves, err_allowed, stop_factor, price_X):
    XA, AB, BC, CD = moves

    AB_range = np.array([.786 - err_allowed, .786 + err_allowed]) * abs(XA)
    BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
    CD_range = np.array([1.618 - err_allowed, 2.618 + err_allowed]) * abs(XA)  # ou abs(BC)

    targets = {
        "XA": None,
        "AB": None,
        "BC": None,
        "CD": None,
        "TP1": None,
        "TP2": None,
        "TP3": None,
        "STOP": None,
    }

    ## === Bullish Butterfly === ##
    if XA > 0 and AB < 0 and BC > 0 and CD < 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D + (price_C - price_D) * 1.618,
                "STOP": price_D - abs(CD) * stop_factor
            })

            return 1, targets  # Bullish

    ## === Bearish Butterfly === ##
    elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
        if AB_range[0] < abs(AB) < AB_range[1] and \
           BC_range[0] < abs(BC) < BC_range[1] and \
           CD_range[0] < abs(CD) < CD_range[1]:

            D = XA + AB + BC + CD
            C = XA + AB + BC
            B = XA + AB

            price_D = price_X + D
            price_C = price_X + C
            price_B = price_X + B

            targets.update({
                "XA": XA,
                "AB": AB,
                "BC": BC,
                "CD": CD,
                "TP1": price_B,
                "TP2": price_C,
                "TP3": price_D - (price_D - price_C) * 1.618,
                "STOP": price_D + abs(CD) * stop_factor
            })

            return -1, targets  # Bearish

    return np.nan, targets


In [ ]:
# def get_crab_hp(moves, err_allowed, stop_factor, price_X):
#     XA, AB, BC, CD = moves
                
#     AB_range = np.array([.382 - err_allowed, .618 + err_allowed]) * abs(XA)
#     BC_range = np.array([.382 - err_allowed, .886 + err_allowed]) * abs(AB)
#     CD_range = np.array([2.24 - err_allowed, 3.618 + err_allowed]) * abs(XA)

#     targets = {
#         "XA": None,
#         "AB": None,
#         "BC": None,
#         "CD": None,
#         "TP1": None,
#         "TP2": None,
#         "TP3": None,
#         "STOP": None,
#     }

#     ## === Bullish Crab === ##
#     if XA > 0 and AB < 0 and BC > 0 and CD < 0:
#         if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:

#             D = XA + AB + BC + CD
#             C = XA + AB + BC
#             B = XA + AB

#             price_D = price_X + D
#             price_C = price_X + C
#             price_B = price_X + B

#             targets.update({
#                 "XA": XA,
#                 "AB": AB,
#                 "BC": BC,
#                 "CD": CD,
#                 "TP1": price_B,
#                 "TP2": price_C,
#                 "TP3": price_D + (price_C - price_D) * 1.618,
#                 "STOP": price_D - abs(CD) * stop_factor
#             })

#             return 1, targets  # Bullish

#     ## === Bearish Crab === ##
#     elif XA < 0 and AB > 0 and BC < 0 and CD > 0:
#         if AB_range[0] < abs(AB) < AB_range[1] and BC_range[0] < abs(BC) < BC_range[1] and CD_range[0] < abs(CD) < CD_range[1]:

#             D = XA + AB + BC + CD
#             C = XA + AB + BC
#             B = XA + AB

#             price_D = price_X + D
#             price_C = price_X + C
#             price_B = price_X + B

#             targets.update({
#                 "XA": XA,
#                 "AB": AB,
#                 "BC": BC,
#                 "CD": CD,
#                 "TP1": price_B,
#                 "TP2": price_C,
#                 "TP3": price_D - (price_D - price_C) * 1.618,
#                 "STOP": price_D + abs(CD) * stop_factor
#             })

#             return -1, targets  # Bearish

#     return np.nan, targets


In [ ]:
# def get_harmonic_pattern(data, err_allowed=0.02, order=5, stop_factor=0.1):
#     close = data.Close
#     fig = go.Figure()

#     # Gráfico principal
#     fig.add_trace(go.Scatter(
#         x=data.Date,
#         y=close,
#         mode='lines',
#         name='Preço',
#         line=dict(color='blue')
#     ))

#     for i in tqdm(range(5, len(close))):
#         current_idx, current_pat, start, end, idx = peak_detect(close.values[:i], order=order)
#         if len(current_pat) < 5:
#             continue

#         XA = current_pat[1] - current_pat[0]
#         AB = current_pat[2] - current_pat[1]
#         BC = current_pat[3] - current_pat[2]
#         CD = current_pat[4] - current_pat[3]
#         moves = [XA, AB, BC, CD]

#         X_price = current_pat[0]
        
#         gart = get_gartley_hp(moves, err_allowed, stop_factor, X_price)
#         bat = get_bat_hp(moves, err_allowed, stop_factor, X_price)
#         butt = get_butterfly_hp(moves, err_allowed, stop_factor, X_price)
#         crab = get_crab_hp(moves, err_allowed, stop_factor, X_price)
        
#         patterns = [gart, butt, bat, crab]
#         labels = ['gart', 'butt', 'bat', 'crab']
        
#         # patterns = [crab]
#         # labels = ['crab']
        

#         for j, pattern in enumerate(patterns):
#             if isinstance(pattern, tuple):
#                 direction, targets = pattern
#                 if np.isnan(direction):
#                     continue

#                 label = ('Bearish ' if direction == -1 else 'Bullish ') + labels[j].capitalize()
#                 color = 'red' if direction == -1 else 'green'

#                 # Desenhar o padrão
#                 fig.add_trace(go.Scatter(
#                     x=data.Date[current_idx],
#                     y=current_pat,
#                     mode='markers+lines+text',
#                     name=label,
#                     marker=dict(size=10, color=color),
#                     text=['X', 'A', 'B', 'C', 'D'],
#                     textposition="top center"
#                 ))

#                 for tp_name in ['TP1', 'TP2', 'TP3']:
#                     tp_price = targets[tp_name]
#                     if tp_price is None:
#                         continue

#                     # Target Lines
#                     fig.add_trace(go.Scatter(
#                         x=[data.Date[current_idx[-1]], data.Date[min(current_idx[-1] + 100, len(data.Date) - 1)]],
#                         y=[tp_price, tp_price],
#                         mode='lines',
#                         name=f'{tp_name} ({label})',
#                         line=dict(color='orange'),
#                         showlegend=True
#                     ))

#                 # === Stop Loss === 
#                 stop_price = targets["STOP"]
#                 if stop_price is not None:
#                     fig.add_trace(go.Scatter(
#                         x=[data.Date[current_idx[-1]], data.Date[min(current_idx[-1] + 100, len(data.Date) - 1)]],
#                         y=[stop_price, stop_price],
#                         mode='lines',
#                         name=f'STOP ({label})',
#                         line=dict(color='white'),
#                         showlegend=True
#                     ))

#             elif pattern == 1 or pattern == -1:
#                 label = ('Bearish ' if pattern == -1 else 'Bullish ') + labels[j].capitalize()
#                 color = 'red' if pattern == -1 else 'green'

#                 fig.add_trace(go.Scatter(
#                     x=data.Date[current_idx],
#                     y=current_pat,
#                     mode='markers+lines+text',
#                     name=label,
#                     marker=dict(size=10, color=color),
#                     text=['X', 'A', 'B', 'C', 'D'],
#                     textposition="top center"
#                 ))

#     fig.update_layout(
#         title="Detecção de Padrão Harmônico com Targets",
#         xaxis_title="Data",
#         yaxis_title="Preço de Fecho",
#         legend_title="Padrões Detectados",
#         template="plotly_dark"
#     )

#     fig.show()


In [ ]:
# def backtest_harmonic_patterns(data, err_allowed=0.02, order=5, stop_factor=0.1, future_window=20):
#     close = data.Close.values
#     results = []

#     for i in tqdm(range(5, len(close) - future_window)):
#         current_idx, current_pat, start, end, idx = peak_detect(close[:i], order=order)
#         if len(current_pat) < 5:
#             continue

#         XA = current_pat[1] - current_pat[0]
#         AB = current_pat[2] - current_pat[1]
#         BC = current_pat[3] - current_pat[2]
#         CD = current_pat[4] - current_pat[3]
#         moves = [XA, AB, BC, CD]
#         price_X = current_pat[0]

#         patterns = [
#             get_gartley_hp(moves, err_allowed, stop_factor, price_X),
#             get_butterfly_hp(moves, err_allowed, stop_factor, price_X),
#             get_bat_hp(moves, err_allowed, stop_factor, price_X),
#             get_crab_hp(moves, err_allowed, stop_factor, price_X)
#         ]

#         pattern_names = ['Gartley', 'Butterfly', 'Bat', 'Crab']

#         valid_patterns = [
#             (name, p[0], p[1])
#             for name, p in zip(pattern_names, patterns)
#             if isinstance(p, tuple) and not np.isnan(p[0])
#         ]

#         if not valid_patterns:
#             continue

#         best = min(valid_patterns, key=lambda x: x[2].get("CD_DIFF", float('inf')))
#         name, direction, targets = best

#         D_index = current_idx[-1]
#         D_price = current_pat[-1]
#         stop_price = targets.get("STOP")
#         future_prices = close[D_index:D_index + future_window]

#         hit_tp = None
#         stop_hit = False

#         for price in future_prices:
#             if direction == 1 and price <= stop_price:
#                 stop_hit = True
#                 break
#             elif direction == -1 and price >= stop_price:
#                 stop_hit = True
#                 break

#             for tp_name in ['TP1', 'TP2', 'TP3']:
#                 tp = targets[tp_name]
#                 if tp is None:
#                     continue

#                 if (direction == 1 and price >= tp) or (direction == -1 and price <= tp):
#                     hit_tp = tp_name
#                     break
#             if hit_tp:
#                 break

#         # ==== Reward/Risk ====
#         reward = None
#         risk = None
#         rr_ratio = None
#         weighted_return = None

#         if hit_tp:
#             reward = abs(targets[hit_tp] - D_price)
#         if stop_price:
#             risk = abs(stop_price - D_price)
#         if reward and risk and risk > 0:
#             rr_ratio = reward / risk

#         if hit_tp:
#             weighted_return = reward
#         elif stop_hit:
#             weighted_return = -risk

#         results.append({
#             "pattern": name,
#             "direction": direction,
#             "hit_tp": hit_tp,
#             "stop_hit": stop_hit,
#             "D_index": D_index,
#             "D_price": D_price,
#             "CD_DIFF": targets.get("CD_DIFF"),
#             "order": order,
#             "err_allowed": err_allowed,
#             "reward": reward,
#             "risk": risk,
#             "rr_ratio": rr_ratio,
#             "weighted_return": weighted_return,
#             "pattern_idx": current_idx,
#             "pattern_idx_dates": data.Date.iloc[current_idx].tolist(),    
#             "pattern_prices": current_pat
#         })

#     return results


In [5]:
# results = backtest_harmonic_patterns(data_history, err_allowed=0.02, order=5, stop_factor=0.1, future_window=20)
# # results

In [ ]:
# def simulate_trading(results, initial_capital=10000, risk_per_trade=0.01):
#     import pandas as pd

#     df = pd.DataFrame(results)
#     df = df.dropna(subset=["risk"]) 

#     capital = initial_capital
#     equity_curve = []
#     trades = []

#     for idx, row in df.iterrows():
#         risk_amount = capital * risk_per_trade

#         if row['risk'] == 0 or pd.isna(row['risk']):
#             continue  # evitar divisões por zero

#         # Tamanho da posição baseado no risco
#         position_size = risk_amount / row['risk']

#         if row['hit_tp']:
#             gain = position_size * row['reward']
#             capital += gain
#             trades.append({"result": "win", "pnl": gain})
#         elif row['stop_hit']:
#             loss = -risk_amount
#             capital += loss
#             trades.append({"result": "loss", "pnl": loss})
#         else:
#             # Nenhum TP ou stop atingido → neutro
#             trades.append({"result": "neutral", "pnl": 0})

#         equity_curve.append(capital)

#     equity_df = pd.DataFrame(trades)
#     equity_df['capital'] = equity_curve

#     return equity_df


In [ ]:
# def normalize_decision_rank(resumo):
#     resumo_norm = resumo.copy()

#     # Garantir que não há NaNs nas colunas-chave antes de normalizar
#     cols_to_normalize = [
#         "capital_final",
#         "expectancy",
#         "reward_risk_medio",
#         "retorno_ponderado_medio",
#         "lucro_por_trade"
#     ]

#     # Preencher NaNs com o mínimo (tratamento conservador)
#     for col in cols_to_normalize:
#         if col in resumo_norm.columns:
#             min_val = resumo_norm[col].min()
#             resumo_norm[col] = resumo_norm[col].fillna(min_val)

#     # Normalizar (0 a 1)
#     scaler = MinMaxScaler()
#     normalized = pd.DataFrame(
#         scaler.fit_transform(resumo_norm[cols_to_normalize]),
#         columns=[col + "_norm" for col in cols_to_normalize]
#     )

#     # Juntar colunas normalizadas
#     resumo_norm = pd.concat([resumo_norm.reset_index(drop=True), normalized], axis=1)

#     # Score ponderado (ajuste pesos conforme a tua estratégia)
#     resumo_norm["score"] = (
#         resumo_norm["capital_final_norm"] * 0.40 +
#         resumo_norm["expectancy_norm"] * 0.25 +
#         resumo_norm["reward_risk_medio_norm"] * 0.15 +
#         resumo_norm["retorno_ponderado_medio_norm"] * 0.15 +
#         resumo_norm["lucro_por_trade_norm"] * 0.05
#     )

#     # Ranking
#     resumo_norm = resumo_norm.sort_values(by="score", ascending=False)
#     resumo_norm["ranking"] = range(1, len(resumo_norm) + 1)

#     return resumo_norm


In [ ]:
# def run_full_backtest(data, future_window=20):
#     """
#     Testa várias combinações de (order, err_allowed, stop_factor) e avalia performance dos padrões harmônicos.
    
#     Return:
#         summary_df: DataFrame com estatísticas de acerto e capital final por combinação
#     """

#     # orders = [2, 3, 4, 5]
#     # err_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
#     # stop_factors = [0.1, 0.2]
#     orders = [5]
#     err_values = [0.05, 0.06]
#     stop_factors = [0.1, 0.2]
#     capex = 10000
#     risk_per_trade=0.01

#     summary = []

#     for order in orders:
#         for err in err_values:
#             for stop in stop_factors:
                
#                 # Roda o backtest
#                 results = backtest_harmonic_patterns(
#                     data,
#                     err_allowed=err,
#                     order=order,
#                     stop_factor=stop,
#                     future_window=future_window
#                 )
#                 df = pd.DataFrame(results)

#                 total = len(df)
#                 hits = df['hit_tp'].notna().sum()
#                 success_rate = hits / total if total > 0 else 0

#                 # Estatísticas de retorno
#                 rr_mean = df['rr_ratio'].mean()
#                 reward_mean = df['reward'].mean()
#                 risk_mean = df['risk'].mean()
#                 weighted_return_mean = df['weighted_return'].mean()

#                 # Simulação de capital
#                 equity_df = simulate_trading(results, initial_capital=capex)
#                 final_capital = equity_df['capital'].iloc[-1] if not equity_df.empty else None

#                 taxa_acerto = round(success_rate * 100, 2)
#                 reward_medio = round(reward_mean, 4) if reward_mean else None
#                 risk_medio = round(risk_mean, 4) if risk_mean else None
#                 reward_risk_medio = round(rr_mean, 4) if rr_mean else None
#                 retorno_ponderado_medio = round(weighted_return_mean, 4) if weighted_return_mean else None
#                 capital_final = round(final_capital, 2) if final_capital else None

#                 expectancy = (
#                     (taxa_acerto / 100) * reward_risk_medio - (1 - taxa_acerto / 100)
#                 ) if reward_risk_medio is not None else None

#                 lucro_por_trade = (
#                     (capital_final - capex) / total if total > 0 else None
#                 )

#                 summary.append({
#                     "order": order,
#                     "err_allowed": err,
#                     "stop_factor": stop,
#                     "padroes_detectados": total,
#                     "targets_atingidos": hits,
#                     "taxa_acerto": taxa_acerto,
#                     "reward_medio": reward_medio,
#                     "risk_medio": risk_medio,
#                     "reward_risk_medio": reward_risk_medio,
#                     "retorno_ponderado_medio": retorno_ponderado_medio,
#                     "expectancy": expectancy,
#                     "lucro_por_trade": lucro_por_trade,
#                     "capital_final": capital_final
#                 })

#     summary_df = pd.DataFrame(summary)
#     summary_df = summary_df.sort_values(by="capital_final", ascending=False)

#     summary_df = normalize_decision_rank(summary_df)

#     best_err_allowed = summary_df.loc[summary_df['ranking']==1, 'err_allowed'].values[0]
#     best_order = summary_df.loc[summary_df['ranking']==1, 'order'].values[0]
#     best_stop_factor = summary_df.loc[summary_df['ranking']==1, 'stop_factor'].values[0]

#     return summary_df



In [ ]:
# resumo = run_full_backtest(data_history)

100%|██████████| 5417/5417 [00:02<00:00, 2181.65it/s]


In [ ]:
# resumo

,order,err_allowed,stop_factor,padroes_detectados,targets_atingidos,taxa_acerto,reward_medio,risk_medio,reward_risk_medio,retorno_ponderado_medio,expectancy,lucro_por_trade,capital_final,capital_final_norm,expectancy_norm,reward_risk_medio_norm,retorno_ponderado_medio_norm,lucro_por_trade_norm,score,ranking
0,5,0.05,0.1,31,10,32.26,0.0147,0.0038,4.5862,0.0019,0.802108,86.037419,12667.16,1.000000,1.00000,1.0,1.000000,1.000000,1.000000,1
1,5,0.06,0.1,33,10,30.30,0.0147,0.0038,4.5862,0.0016,0.692619,73.184545,12415.09,0.903544,0.86387,1.0,0.884615,0.847725,0.902464,2
2,5,0.05,0.2,31,10,32.26,0.0147,0.0076,2.2931,-0.0003,0.062354,8.321613,10257.97,0.078111,0.08025,0.0,0.153846,0.079261,0.078347,3
3,5,0.06,0.2,33,10,30.30,0.0147,0.0076,2.2931,-0.0007,-0.002191,1.631515,10053.84,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,4


In [ ]:
# best_err_allowed = resumo.loc[resumo['ranking']==1, 'err_allowed'].values[0]
# best_order = resumo.loc[resumo['ranking']==1, 'order'].values[0]
# best_stop_factor = resumo.loc[resumo['ranking']==1, 'stop_factor'].values[0]

In [6]:
# get_harmonic_pattern(data_history, err_allowed=best_err_allowed, order=best_order, stop_factor=best_stop_factor)

In [7]:
# https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


# ### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
import requests

url = "https://api.binance.com/api/v3/ticker/price"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)

if response.status_code == 200:
    symbol = response.json()["symbol"]
    price = response.json()["price"]
else:
    error = response.status_code

In [ ]:
from decimal import Decimal
from datetime import datetime


def get_crypto_symbol_24h(symbol : str):

    url = "https://api.binance.com/api/v3/ticker/24hr"
    params = {"symbol": symbol}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        priceChangePercent = Decimal(response.json()["priceChangePercent"])
        weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
        prevClosePrice = Decimal(response.json()["prevClosePrice"])
        priceChange = Decimal(response.json()["priceChange"])
        lastPrice = Decimal(response.json()["lastPrice"])
        lastQty = Decimal(response.json()["lastQty"])
        bidPrice = Decimal(response.json()["bidPrice"])
        bidQty = Decimal(response.json()["bidQty"])
        askPrice = Decimal(response.json()["askPrice"])
        askQty = Decimal(response.json()["askQty"])
        openPrice = Decimal(response.json()["openPrice"])
        highPrice = Decimal(response.json()["highPrice"])
        lowPrice = Decimal(response.json()["lowPrice"])
        volume = Decimal(response.json()["volume"])
        quoteVolume = Decimal(response.json()["quoteVolume"])
        openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        firstId = response.json()["firstId"]
        lastId = response.json()["lastId"]
        count = response.json()["count"]
    else:
        print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
url = "https://api.binance.com/api/v3/openOrders"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)
response.json()
